#Imports


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import random
import zipfile
import pathlib
import os
import itertools

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns

split-folders PyPI: Split folders with files (e.g. images) into train, validation and test (dataset) folders.

In [2]:
!pip install split-folders
import splitfolders

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp '/content/drive/MyDrive/Datasets/COVID-19_Radiography_Dataset.zip' .
!unzip -q -n COVID-19_Radiography_Dataset.zip
data_dir = '/content/COVID-19_Radiography_Dataset'
#Dataset path = /content/COVID-19_Radiography_Dataset

In [5]:
# Setup some global variables
BATCH_SIZE = 64
IMG_SIZE = (299,299)

#2.1 Συνάρτηση prepare_datasets

In [6]:
def prepare_datasets(data_dir, train_pct=0.6, val_pct=0.2, test_pct=0.2, batch_size=64, img_size=(299, 299)):
    splitfolders.ratio(data_dir, output="/content/Data", ratio=(train_pct, val_pct, test_pct))

    # Get the subdirectories (classes)
    data_dir = pathlib.Path('/content/Data/test')
    classes = np.array(sorted(item.name for item in data_dir.glob("*")))

    # Test train validation split
    test_dir = '/content/Data/test'
    train_dir = '/content/Data/train'
    val_dir = '/content/Data/val'

    test_datagen = ImageDataGenerator(rescale = 1/255.)
    train_datagen = ImageDataGenerator(rescale = 1/255.)
    valid_datagen = ImageDataGenerator(rescale = 1/255.)

    test_ds = tf.keras.utils.image_dataset_from_directory(directory=test_dir,
                                                          color_mode = 'grayscale',
                                                          batch_size = BATCH_SIZE,
                                                          image_size = IMG_SIZE,
                                                          label_mode = 'categorical')

    train_ds = tf.keras.utils.image_dataset_from_directory(directory=train_dir,
                                                           color_mode = 'grayscale',
                                                           batch_size = BATCH_SIZE,
                                                           image_size = IMG_SIZE,
                                                           label_mode = 'categorical')

    val_ds = tf.keras.utils.image_dataset_from_directory(directory=val_dir,
                                                         color_mode = 'grayscale',
                                                         batch_size = BATCH_SIZE,
                                                         image_size = IMG_SIZE,
                                                         label_mode = 'categorical')

    devel_ds = np.concatenate((train_ds, val_ds), axis=None)

    return devel_ds, train_ds, val_ds, test_ds, classes

#3 Απλό συνελικτικό δίκτυο

In [7]:
devel_ds, train_ds, val_ds, test_ds, class_names = prepare_datasets(data_dir="/content/COVID-19_Radiography_Dataset")

Copying files: 21165 files [00:09, 2206.43 files/s]


Found 4235 files belonging to 4 classes.
Found 12698 files belonging to 4 classes.
Found 4232 files belonging to 4 classes.


In [8]:
def cnn1(num_classes):
    model = tf.keras.Sequential([
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(8,3,activation="relu",input_shape = (299,299,3)),
      tf.keras.layers.MaxPooling2D(strides = 2),
      tf.keras.layers.Conv2D(16,3,activation="relu",input_shape = (299,299,3)),
      tf.keras.layers.MaxPooling2D(strides = 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(num_classes,activation="softmax")
    ])
    return model

In [9]:
model_1 = cnn1(num_classes=4)

In [ ]:
# Compile Model 1
model_1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 10**-3,
                                                     beta_1 = 0.9,
                                                     beta_2 = 0.99 ),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ["accuracy"])

# Setup early callback
callback = tf.keras.callbacks.EarlyStopping(monitor ="loss",patience = 3)

# Fit the model
history_1 = model_1.fit(train_ds,
            batch_size = BATCH_SIZE,
            steps_per_epoch = len(train_ds),
            validation_data = val_ds,
            validation_steps = len(val_ds),
            epochs = 20,
            callbacks = [callback])

Epoch 1/20
199/199 [==============================] - 59s 224ms/step - loss: 1.0773 - accuracy: 0.6661 - val_loss: 0.6983 - val_accuracy: 0.7339
Epoch 2/20
199/199 [==============================] - 42s 208ms/step - loss: 0.5343 - accuracy: 0.7948 - val_loss: 0.5002 - val_accuracy: 0.8086


# 3.2 Αξιολόγηση

In [ ]:
def create_and_plot_confusion(model, dataset, class_names = class_names, pre_trained = False):
    # Get predictions and labels
    if pre_trained:
        y_pred = model.predict(dataset)
        pred = np.argmax(y_pred, axis = 1)
        labels = np.argmax(dataset, axis = 1)
    else:
        pred = np.array([])
        labels = np.array([])
        for x , y in dataset:
            pred = np.concatenate([pred, np.argmax(model.predict(x), axis=-1)])
            labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

    # Create a confusion matrix
    confusion =  tf.math.confusion_matrix(labels=labels, predictions=pred).numpy()

    # Create multi class color plot for confusion matrix
    label_conf = np.arange(confusion.shape[0])
    fig , ax = plt.subplots(figsize=(16,16))
    cax = ax.matshow(confusion, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    ax.set(title="Confusion Matrix",
           xlabel="Predicted",
           ylabel="True",
           xticks = np.arange(len(class_names)),
           yticks = np.arange(len(class_names)),
           xticklabels = class_names,
           yticklabels = class_names)
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    # Set the threshold for different colors
    threshold = (confusion.max() + confusion.min()) / 2.

    for i, j in itertools.product(range(confusion.shape[0]), range(confusion.shape[1])):
        plt.text(j, i, f"{confusion[i, j]}",
                horizontalalignment="center",
                color="white" if confusion[i, j] > threshold else "black",
                size=14)

In [ ]:
print(f" Training :{model_1.evaluate(train_ds)}")
print(f" Testing  :{model_1.evaluate(test_ds)}")
print(f" Valid    :{model_1.evaluate(val_ds)}")

In [ ]:
create_and_plot_confusion(model_1, test_ds)

# 4 Συνελικτικό δίκτυο μεγαλύτερου βάθους

In [ ]:
def cnn2(num_classes):
    model = tf.keras.models.Sequential([
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(32,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.Conv2D(32,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.MaxPooling2D(strides = 4),

      tf.keras.layers.Conv2D(64,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.Conv2D(64,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.MaxPooling2D(strides = 2),

      tf.keras.layers.Conv2D(128,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.Conv2D(128,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.MaxPooling2D(strides = 2),

      tf.keras.layers.Conv2D(256,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.Conv2D(256,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.Conv2D(256,3,input_shape = (224,224,3),activation="relu"),
      tf.keras.layers.MaxPooling2D(strides = 2),

      tf.keras.layers.Conv2D(512, 3,input_shape = IMG_SIZE,activation="relu"),
      tf.keras.layers.MaxPooling2D(2),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1024, activation="relu"),
      tf.keras.layers.Dense(num_classes,activation="softmax")
    ])

    return model

In [ ]:
# Create the model
model_2 = cnn2(len(class_names))

# Compile Model 2
model_2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 10**-3,
                                                     beta_1 = 0.9 ,
                                                     beta_2 =0.99 ),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ["accuracy"])

# Setup early callback
callback = tf.keras.callbacks.EarlyStopping(monitor ="loss",patience = 3)

# Fit the model
history_2 = model_2.fit(train_ds,
                        steps_per_epoch = len(train_ds),
                        epochs = 20,
                        batch_size = BATCH_SIZE,
                        validation_data = val_ds,
                        validation_steps = len(val_ds),
                        callbacks = [callback])

In [ ]:
print(f" Training :{model_2.evaluate(train_ds)}")
print(f" Testing  :{model_2.evaluate(test_ds)}")
print(f" Valid    :{model_2.evaluate(val_ds)}")

In [ ]:
create_and_plot_confusion(model_2, test_ds)

# 5 Με χρήση προεκπαιδευμένου δικτύου

In [ ]:
# Change global variables
BATCH_SIZE = 32
IMG_SIZE = (224,244)

# Test train validation split
train_dir = '/content/Data/train'
test_dir = '/content/Data/test'
val_dir = '/content/Data/val'

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)
valid_datagen = ImageDataGenerator(rescale = 1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                                  target_size = IMG_SIZE,
                                                  batch_size = BATCH_SIZE,
                                                  class_mode = "categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                                target_size = IMG_SIZE,
                                                batch_size = BATCH_SIZE,
                                                class_mode = "categorical")

val_data = valid_datagen.flow_from_directory(val_dir,
                                                target_size = IMG_SIZE,
                                                batch_size = BATCH_SIZE,
                                                class_mode = "categorical")

In [ ]:
model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# Create the efficientnet layer
feature_extractor_layer  = hub.KerasLayer(model_url,
                               trainable=False,
                               input_shape = IMG_SIZE+(3,))

output_size = len(class_names)

# Now time to create our own model
model_3  = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(output_size,activation="softmax")
])

# Compile Model 3
model_3.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 10**-3,
                                                     beta_1 = 0.9 ,
                                                     beta_2 =0.99 ),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ["accuracy"])

# Setup early callback
callback = tf.keras.callbacks.EarlyStopping(monitor ="loss",patience = 3)

# Fit the model
history_3 = model_3.fit(train_data,
                        epochs = 5,
                        batch_size = BATCH_SIZE,
                        validation_data = val_data,
                        callbacks = [callback])

In [ ]:
print(f" Training :{model_3.evaluate(train_data)}")
print(f" Testing  :{model_3.evaluate(test_data)}")
print(f" Valid    :{model_3.evaluate(val_data)}")

In [ ]:
create_and_plot_confusion(model_3, test_data, pre_trained = True)